In [13]:
import numpy as np
import pandas as pd 
import itertools
from collections import Counter

In [14]:
UNK = "<UNK>"

def handle_unknown(word,vocab):
    if word not in vocab:
        return UNK
    return word

In [15]:
#brute force way 
# if sentences are long there will be many possible tag sequences and overall many iterations to check so we use viterbi algorithm to optimize


def sequence_probability(words, tag_seq,initial_probs,transition_probs,emission_probs):
#for computing probabilities of a tag sequence in brute force way
    prob = 1.0

    word = words[0]
    prob *= initial_probs.get(tag_seq[0], 0)
    prob *= emission_probs.get((tag_seq[0], word), 0)

    # example for a three letter word
    
    # P(t1)×P(w1∣t1)×P(t2∣t1)×P(w2∣t2)×P(t3∣t2)×P(w3∣t3)
    # p(t1)-initial_prob

    for i in range(1, len(words)):
        prev_tag = tag_seq[i-1]
        curr_tag = tag_seq[i]

        word = words[i]

        prob *= transition_probs.get((prev_tag, curr_tag), 0)
        prob *= emission_probs.get((curr_tag, word), 0)

    return prob


def brute_force(words,all_tags,initial_probs,transition_probs,emission_probs):
    words = [handle_unknown(word) for word in words]
    best_prob = -1
    best_seq = None

    # generate probability for every possible tag sequence
    for tag_seq in itertools.product(all_tags, repeat=len(words)):

        prob = sequence_probability(words,tag_seq,initial_probs,transition_probs,emission_probs)

        if prob > best_prob:
            best_prob = prob
            best_seq = tag_seq

    return list(best_seq)

In [16]:
def viterbi(words,all_tags,initial_probs,transition_probs,emission_probs,vocab):
    words = [handle_unknown(word,vocab) for word in words]
    T = len(words)
    N = len(all_tags)
    
    V = {}
    B = {}

    for tag in all_tags:
        V[(tag, 0)] = (
            initial_probs.get(tag, 0) *
            emission_probs.get((tag, words[0]), 0)
        )
        B[(tag, 0)] = None
        
    # Vt(tagj ) = max(tagi(Vt−1(tagi) × P(tagj | tagi) × P(wordt| tagj ))
    
    for i in range(1, len(words)):
        for curr_tag in all_tags:

            best_prob = 0
            best_prev = None

            for prev_tag in all_tags:
                prob = (V[(prev_tag, i-1)] *transition_probs.get((prev_tag, curr_tag), 0) *emission_probs.get((curr_tag, words[i]), 0))
                
                if prob > best_prob:
                    best_prob = prob
                    best_prev = prev_tag
            V[(curr_tag, i)] = best_prob
            B[(curr_tag, i)] = best_prev
                #backtracking matric stores the best prev tag (tag that leads to the maximum Viterbi probability)s at the particular position for a particular tag
    
    #to choose which final tag gives the highest probability overall
    
    last_pos = len(words) - 1    
    # Return the item for which V[(t, last_pos) is largest

    best_last_tag = None
    best_last_prob = -1

    for tag in all_tags:
        if V[(tag, last_pos)] > best_last_prob:
            best_last_prob = V[(tag, last_pos)]
            best_last_tag = tag

    # the viterbi maxtrix contain the max probability that can be obtained with a specific tag sequence that ends with that tag at that position.
    # we backtrack from the final  tag with highest probability
    best_tags = [best_last_tag]

    for i in range(last_pos, 0, -1):
        prev = B.get((best_tags[-1], i))

        if prev is None:
            # fallback: choose tag with max probability at previous position
            prev = max(
                all_tags,
                key=lambda t: V[(t, i-1)]
            )

        best_tags.append(prev)

    best_tags.reverse()


    
    return best_tags